# Evaluate sea ice in area of interest 
This notebook introduces a function that runs mean and anomaly calculations on sea-ice concentration (sic) data for the Amundsen Sea Embayment (ASE) in West Antarctica. The sic datum used are the [AMSR-E/Aqua](https://nsidc.org/data/ae_si6/versions/3) and [AMSR-E/AMSR2](https://nsidc.org/data/ae_si6/versions/3) products from the [National Snow and Ice Data Center](https://nsidc.org/home). 

In [ ]:
# import modules
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle
import numpy.ma as ma 
from pyproj import Transformer
import xarray as xr
import cmocean
import time 

Load in SIC data and Bedmachine for plotting. MUST **insert file paths!**

In [ ]:
# Load sea-ice data 
with np.load('ase_sic_12k.npz') as infile:
    data = np.ma.MaskedArray(data=infile['data'], mask=infile['mask'])
    psx = infile['x']
    psy = infile['y']
    dates = infile['dates']

# Modify sea ice data
# set values 110, 120 to NaN
row, col, depth = data.shape
sic = np.empty((row, col, depth))
sic[:] = np.nan

for i in range(0, depth):
        day = data[:,:,i]
        day[day == 110] = 'NaN' # masks missing data
        day[day == 120] = 'NaN' # masks land 
        day = day 
        sic[:,:,i] = day


# Load BedMachine Antarctica V2 and extract ice masks
# insert path to the BedMachine netCDF file, ends in .nc
bmpath = r'INSERT PATH'

# open the netCDF file
xr_ds = xr.open_dataset(bmpath)

mask = xr_ds.mask.squeeze()

# convert ocean mask from 0 to nan
mask.data = mask.data.astype('float')
mask.data[mask.data == 0] = 'nan'
    
# Downsample and extract mask for ASE from BedMachine
# find and round the extent of the ASE sic data to the nearest 1 km
xmin = np.floor(min(psx)/1000)*1000
xmax = np.round(max(psx)/1000)*1000
ymin = np.floor(min(psy)/1000)*1000
ymax = np.round(max(psy)/1000)*1000

dxy = 2500 #interpolation interval, in meters 
dxy_sic = 12500 #resolution of sic data, in meters 

# x and y coordinates for interpolating the mask
xx = np.arange(xmin-dxy_sic/2, xmax+dxy_sic/2, dxy)
yy = np.arange(ymin-dxy_sic/2, ymax+dxy_sic/2, dxy)

# interpolate the mask
mask_int = mask.interp(coords={'x': xx,'y': yy}, method = 'linear')

In [ ]:
# OPTIONAL: plot just bedmachine, for inspection  
plt.figure(dpi = 200)
plt.pcolormesh(mask_int.x/1000, mask_int.y/1000, mask_int.data, cmap=cmocean.cm.gray, shading='nearest', vmin=1, vmax=4)
plt.xlabel('PSX (km)')
plt.ylabel('PSY (km)')
plt.axis('equal')
plt.show()

In [ ]:
# define sea-ice function for monthly averages and anomalies 
def get_sea_ice_stats(x1,y1,x2,y2, sic_arr, dates_arr, psx_arr, psy_arr, day_plot):
    """
    Input PSXY coordinates, in km, of top left and bottom right corners of area of interest. Coordinates must be within the 
    ASE. Input sic, dates, and PSXY arrays and choice of day to plot. Outputs one-day plot of interest area over regional plot
    of the ASE and plots of the area's monthly average sea-ice concentration, monthly average sea-ice area, and monthly sea-ice
    area anomaly.
    """ 
    start = time.time()
    
    ## Find index of PSXY coordinates 
    arr_x = psx_arr
    arr_y = psy_arr
    
    xa = x1 * 1000
    diff_xa = np.absolute(arr_x - xa)
    idx_x1 = diff_xa.argmin()
    
    ya = y1 * 1000
    diff_ya = np.absolute(arr_y - ya)
    idx_y1 = diff_ya.argmin()
     
    xb = x2 * 1000 
    diff_xb = np.absolute(arr_x - xb)
    idx_x2 = diff_xb.argmin()
    
    yb = y2 * 1000
    diff_yb = np.absolute(arr_y - yb)
    idx_y2 = diff_yb.argmin()
    
    xs = [idx_x1, idx_x2]
    ys = [idx_y1, idx_y2]
    
    area_sic = sic_arr[min(ys):max(ys)+1, min(xs):max(xs)+1, :]
    row, col, depth = area_sic.shape # depth = number of days 
    
    # get psxy for area of interest for plotting 
    area_x = psx_arr[min(xs):max(xs)+1]
    area_y = psy_arr[min(ys):max(ys)+1]
     
    ## Output array 1 calculation 
    # find monthly averages of sea-ice concentration
    
    monthly_sic_avgs = [] # output array 1 
    month_labels = []
    
    for j in range(min(dates_arr[:,0]), max(dates_arr[:,0])+1):
                   for i in range(1,13):
                       idx = np.argwhere((dates_arr[:,0] == j) & (dates_arr[:,1] == i))
                       avg = np.nanmean(area_sic[:,:,idx])
                       label = str(j) + '/' + str(i)
                       monthly_sic_avgs = np.append(monthly_sic_avgs,avg)
                       month_labels = np.append(month_labels, label)
    
    # get sea-ice area from sea-ice concentration
    area_sia = np.empty((row, col, depth))
    area_sia[:] = np.nan

    for i in range(depth):
        area_sia[:,:,i] = area_sic[:,:,i]/100 * 12.5**2

    ## Output array 2 calculation 
    # find monthly average sea-ice area  
    
    monthly_sia_avgs = [] # output array 2
    
    months = [] # create list of months
    years = [] # create list of years 
    
    for j in range(min(dates_arr[:,0]), max(dates_arr[:,0])+1):
        for i in range(1,13):
            idx = np.argwhere((dates_arr[:,0] == j) & (dates_arr[:,1] == i))
            total = np.nansum(np.nansum(area_sia[:,:,idx]))
            days = len(idx)
            avg = total / days
            monthly_sia_avgs = np.append(monthly_sia_avgs, avg)
            year = int(j)
            years = np.append(years, year)
            month = int(i)
            months = np.append(months, month)
    
    dates_monthly = np.vstack((years,months)) # creates new dates array that is just months, not daily. output array 4  
    
    ## Output array 3 calculation 
    # find series average of monthly sums for each month
    
    series_avgs = [0] #set first element be zero to simplify indexing for future anomaly calculation
    
    for i in range(1,13):
        idx = np.argwhere((dates_monthly[1] == i))
        avg = np.nanmean(monthly_sia_avgs[idx])
        series_avgs = np.append(series_avgs, avg)

    # calculate monthly total sia anomaly 
    monthly_sia_anoms = [] # output array 3 
    
    year_min = min(dates_monthly[0,:], key=lambda x:float(x))
    year_max = max(dates_monthly[0,:], key=lambda x:float(x))
    
    for j in range(int(year_min), int(year_max)+1):
        for i in range(1,13):
            idx = np.argwhere((dates_monthly[0] == j) & (dates_monthly[1] == i))
            anom = monthly_sia_avgs[idx] - series_avgs[i]
            monthly_sia_anoms = np.append(monthly_sia_anoms, anom)
    
    ## Plot interest area and its monthly avgs (sic), totals (sia), and anomalies (sia)
    
    # index of a day to plot
    area_plot = area_sic[:, :, day_plot]
    ase_plot = sic_arr[:, :, day_plot]
    
    # rectangle for area of interest 
    width = (psx_arr[idx_x2]/1000) - (psx_arr[idx_x1]/1000)
    height = (psy_arr[idx_y1]/1000) - (psy_arr[idx_y2]/1000)
    
    # plot one day
    plt.figure(dpi = 200)
    plt.pcolormesh(psx_arr/1000, psy_arr/1000, ase_plot, cmap=cmocean.cm.ice, shading='nearest', vmin=0, vmax=100)
    cbar = plt.colorbar()
    cbar.set_label('Sea-ice concentration (%)')
    plt.pcolormesh(mask_int.x/1000, mask_int.y/1000, mask_int.data, cmap=cmocean.cm.gray, shading='nearest', vmin=1, vmax=4)
    plt.xlabel('PSX (km)')
    plt.ylabel('PSY (km)')
    plt.gca().add_patch(Rectangle((psx_arr[idx_x1]/1000,psy_arr[idx_y2]/1000), width, height, edgecolor='red', fill=False))
    titletxt = str(dates[day_plot,0]) + '/' + str(dates[day_plot,1]) + '/' + str(dates[day_plot,2])
    plt.title(titletxt)
    plt.axis('equal')
    plt.show()
                                    
    # plot avgs and anomalies 
    length = len(monthly_sic_avgs)
    x = month_labels[0:length+1:6] # labels for x-axis for avg concentration plot
    zeros = np.zeros(length) # for zero slope in anomaly plot
    
    fig, (ax0, ax1, ax2) = plt.subplots(3,1, figsize=(20,20))
    fig.tight_layout(h_pad = 5)
    ax0.plot(month_labels,monthly_sic_avgs, 'darkorange')
    ax0.set_xticks(np.arange(1,length+1,6))
    ax0.set_xticklabels(x,rotation=60)
    ax0.set(title=r'Monthly average sea-ice concentration', ylabel=r'SIC (%)')
    
    ax1.plot(month_labels, monthly_sia_avgs, 'seagreen')
    ax1.set_xticks(np.arange(1,length+1,6))
    ax1.set_xticklabels(x,rotation=60)
    ax1.set(title=r'Monthly average sea-ice area', ylabel=r'SIA ($km^2$)')
    
    ax2.plot(month_labels, monthly_sia_anoms, 'darkviolet')
    ax2.plot(month_labels, zeros, 'k--')
    ax2.set_ybound(upper= np.nanmax(abs(monthly_sia_anoms))+1000, lower=-np.nanmax(abs(monthly_sia_anoms))-1000)
    ax2.set_xticks(np.arange(1, length+1,6))
    ax2.set_xticklabels(x, rotation=60)
    ax2.set(title=r'Monthly sea-ice area anomaly', ylabel=r'SIA ($km^2$)')
    plt.show()
    
    end = time.time()
    print('Elapsed time:', np.round((end-start)*1e6)/1e6, 'seconds.')
    
    return monthly_sic_avgs, monthly_sia_avgs, monthly_sia_anoms, dates_monthly, month_labels
    # optional: save arrays as an npz file
   # np.savez_compressed('interest_area_data.npz', monthly_sic_avgs = monthly_sic_avgs, monthly_sia_avgs = monthly_sia_avgs, 
    #                    monthly_sia_anomaly = monthly_sia_anoms, dates_monthly = dates_monthly)

In [ ]:
# sea-ice function for seasonal (summer, winter) averages and anomalies
def get_sia_seasonal_anomalies(x1, y1, x2, y2, sic_arr, dates_arr, psx_arr, psy_arr):
    """
    Input PSXY coordinates, in km, of top left and bottom right corners of area of interest in the Amundsen Sea Embayment.\
    Output plot showing interest area in ASE, lists of summer and winter anomalies, and time labels for each season. 
    """
    start = time.time()
    
    ## find index of PSXY coordinates 
    xa = x1 * 1000
    diff_xa = np.absolute(psx_arr - xa)
    idx_x1 = diff_xa.argmin()
    
    ya = y1 * 1000
    diff_ya = np.absolute(psy_arr - ya)
    idx_y1 = diff_ya.argmin()
    
    xb = x2 * 1000
    diff_xb = np.absolute(psx_arr - xb)
    idx_x2 = diff_xb.argmin()
    
    yb = y2 * 1000
    diff_yb = np.absolute(psy_arr - yb)
    idx_y2 = diff_yb.argmin()
    
    xs = [idx_x1, idx_x2]
    ys = [idx_y1, idx_y2]
    
    area = sic_arr[min(ys):max(ys)+1, min(xs):max(xs)+1, :]
    row, col, depth = area.shape 
    
    # get psxy of area of interest for plotting 
    area_x = psx_arr[min(xs):max(xs)+1]
    area_y = psy_arr[min(ys):max(ys)+1]
    
    # set error 
    sic_error = 2
    sia_error = (sic_error/100)*12.5**2
    
    # get sea-ice area from sea-ice concentration 
    area_sia = np.empty((row, col, depth))
    area_sia[:] = np.nan 
    for i in range(depth):
        area_sia[:,:,i] = area[:,:,i]/100*12.5**2
    
    # get error for area of sea ice 
    area_error = (row*col)*sia_error 
    
    # get summer sea-ice area averages 
    summer_avgs = []
    summer_uncertainty = []
    summer_labels = []
    for j in range(min(dates_arr[:,0]), max(dates_arr[:,0])+1):
        if j == 2011:
            avg = np.nan
            summer_avgs = np.append(summer_avgs, avg)
            uncert = np.nan
            summer_uncertainty = np.append(summer_uncertainty, uncert)
            label = str(int(j)) + '-' + str(int(j+1))
            summer_labels = np.append(summer_labels, label)
        else: 
            idx = np.argwhere((dates_arr[:,1]>=10)&(dates_arr[:,0]==j)|(dates_arr[:,1]<=3)&(dates[:,0]==j+1))
            total = np.nansum(np.nansum(area_sia[:,:,idx]))
            avg = total / len(idx) 
            summer_avgs = np.append(summer_avgs, avg)
            n = int(len(idx))
            uncert = area_error/np.sqrt(n)
            summer_uncertainty = np.append(summer_uncertainty,uncert)
            label = str(int(j)) + '-' + str(int(j+1))
            summer_labels = np.append(summer_labels, label)
    summer_avgs = summer_avgs[:20] #cut off last avg bc data goes to 05/2022, no 2022-23 season 
    summer_uncertainty = summer_uncertainty[:20]
    summer_labels = summer_labels[:20]
    
    # get summer anomalies 
    summer_series_avg = np.nanmean(summer_avgs)
    
    summer_anoms = []
    for i in range(len(summer_avgs)):
        anom = summer_avgs[i] - summer_series_avg
        summer_anoms = np.append(summer_anoms, anom)
    
    # get winter sea-ice area averages
    winter_monthly_avgs = [] 
    years = [] 
    
    for j in range(2003, max(dates_arr[:,0])+1):
        for i in range(4,9):
            idx = np.argwhere((dates_arr[:,1]==i)&(dates_arr[:,0]==j))
            total = np.nansum(np.nansum(area_sia[:,:,idx]))
            avg = total / len(idx)
            winter_monthly_avgs = np.append(winter_monthly_avgs, avg)
            years = np.append(years, int(j))
    
    winter_avgs = []
    winter_labels = [] 
    
    for j in range(2003, 2023):
        if j == 2011:
            avg = np.nan
            winter_avgs = np.append(winter_avgs, avg)
            winter_labels = np.append(winter_labels, str(j))
        elif j == 2012:
            avg = np.nan 
            winter_avgs = np.append(winter_avgs, avg)
            winter_labels = np.append(winter_labels, str(j))
        else: 
            idx = np.argwhere((years == j))
            total = np.nansum(winter_monthly_avgs[idx])
            avg = total/5
            winter_avgs = np.append(winter_avgs, avg)
            winter_labels = np.append(winter_labels, str(j))
    
    winter_avgs = winter_avgs[:19]
    winter_labels = winter_labels[:19]
    
    # get winter anomalies 
    winter_series_avg = np.nanmean(winter_avgs)
    
    winter_anoms = []
    for i in range(len(winter_avgs)):
        anom = winter_avgs[i] - winter_series_avg
        winter_anoms = np.append(winter_anoms, anom)
        
    ## plot interest area and its anomalies 
    area_plot = area[:,:, 10]
    ase_plot = sic_arr[:,:,10]
    
    # rectangle-area of interest 
    width = (psx_arr[idx_x2]/1000) - (psx_arr[idx_x1]/1000) 
    height = (psy_arr[idx_y1]/1000) - (psy_arr[idx_y2]/1000) 
    
    plt.figure(dpi = 200)
    plt.pcolormesh(psx_arr/1000, psy_arr/1000, ase_plot, cmap=cmocean.cm.ice, shading='nearest', vmin=0, vmax=100)
    cbar = plt.colorbar()
    cbar.set_label('Sea-ice concentration [%]')
    plt.pcolormesh(mask_int.x/1000, mask_int.y/1000, mask_int.data, cmap=cmocean.cm.gray, shading='nearest', vmin=1, vmax=4)
    plt.xlabel('PSX [km]')
    plt.ylabel('PSY [km]')
    plt.gca().add_patch(Rectangle((psx_arr[idx_x1]/1000,psy_arr[idx_y2]/1000), width, height, edgecolor='red', fill=False))
    plt.axis('equal')
    plt.show()
    
    # plot anomalies 
    summer_zeros = np.zeros(len(summer_avgs))
    winter_zeros = np.zeros(len(winter_avgs))
    
    plt.figure(dpi = 200)
    plt.plot(summer_labels, summer_anoms, 'orangered')
    plt.plot(summer_labels, summer_zeros, 'k--')
    plt.xticks(rotation=90)
    plt.ylim(top=np.nanmax(abs(summer_anoms))+1000, bottom=-np.nanmax(abs(summer_anoms))-1000)
    plt.ylabel('Sea-ice area [$km^2$]')
    plt.title('Summer sea-ice area anomaly')
    plt.show()
    
    plt.figure(dpi=200)
    plt.plot(winter_labels, winter_anoms, 'midnightblue')
    plt.plot(winter_labels, winter_zeros, 'k--')
    plt.xticks(rotation=90)
    plt.ylim(top=np.nanmax(abs(winter_anoms))+1000, bottom=-np.nanmax(abs(winter_anoms))-1000)
    plt.ylabel('Sea-ice area [$km^2$]')
    plt.title('Winter sea-ice area anomaly')
    plt.show()
    
    #return summer_anoms, summer_labels,winter_anoms, winter_labels
    return summer_avgs, summer_uncertainty, summer_anoms,summer_labels

### Test functions 
Use polar coordinates of Pine Island Bay to test function: <br> 
>Top-left corner: (-1800, -325) <br>
>Bottom-right corner: (-1500, -600) 

In [ ]:
# test monthly function
pib_sic_avgs,pib_sia_avgs,pib_sia_anoms,months_dates,month_labels = get_sea_ice_stats(-1800, -325, -1500, -600, sic, 
                                                                                      dates, psx, psy, 10)

In [ ]:
# test seasonal function
a,b,c,d= get_sia_seasonal_anomalies(-1800, -325, -1500, -600, sic, dates, psx, psy)